## BIKE

### First test on endomondo_sample db

### Rearrange so workoutid clustered

```
cluster bike using bike_workoutid_idx;
```

### REMOVE DUPES.

#### Count the number of dupes

```
select count(*) from (select count(*) from bike group by altitude, heart_rate, latitude, longitude, speed, workoutid, time having count(*) > 1) as foo;
```
```
 count 
-------
  1163
```

#### Remove the dupes

```
ALTER TABLE bike ADD COLUMN id SERIAL PRIMARY KEY;
DELETE FROM bike
WHERE id IN (SELECT id
              FROM (SELECT id,
                             ROW_NUMBER() OVER (partition BY altitude, heart_rate, latitude, longitude, speed, workoutid, time ORDER BY id) AS rnum
                     FROM bike) t
              WHERE t.rnum > 1);
ALTER TABLE bike drop column id;
```
```
DELETE 1163
```

```
select count(*) from (select count(*) from bike group by altitude, heart_rate, latitude, longitude, speed, workoutid, time having count(*) > 1) as foo;
```

```
 count 
-------
     0
```

### Vacuum
```
vacuum full bike;
```

## Speed

### Value given in kph. Range should be from 0-125.

```
select * from histogram('speed', 'bike');
```
```
 bucket |              range              |  freq   |       bar       
--------+---------------------------------+---------+-----------------
      1 | [0.0000000000,11.3904000000]    |  202785 | **
      2 | [11.3940000000,22.7808000000]   |  811894 | *******
      3 | [22.7830000000,34.1712000000]   | 1654171 | ***************
      4 | [34.1748000000,45.5616000000]   |  500694 | *****
      5 | [45.5652000000,56.9520000000]   |   57235 | *
      6 | [56.9556000000,68.3244000000]   |    5769 | 
      7 | [68.3496000000,79.6248000000]   |     563 | 
      8 | [79.8084000000,91.0656000000]   |     102 | 
      9 | [91.2492000000,102.4490000000]  |      59 | 
     10 | [103.1470000000,113.3140000000] |     146 | 
     11 | [113.9800000000,124.4020000000] |      50 | 
     12 | [133.2790000000,135.4750000000] |       4 | 
     14 | [156.4520000000,158.1410000000] |       2 | 
     15 | [167.4900000000,167.4900000000] |       1 | 
     16 | [179.1360000000,179.1360000000] |       1 | 
     17 | [182.4950000000,182.6750000000] |       2 | 
     18 | [194.1520000000,194.1520000000] |       1 | 
     20 | [226.5440000000,226.5440000000] |       1 | 
     21 | [227.8120000000,227.8120000000] |       1 | 
```

#### Simply removing data has implications, there is a relatively small amount of outliers so we will just smooth them.

#### Lets find the average speed and smooth them to that.

```
SELECT avg(speed) AS average FROM bike where speed < 125 and speed > 0;
```
```
       average       
---------------------
 26.7493104169393044
```

### Still too many > 125kph (80mph, tough to go that fast).

```
update bike set speed = 26.749 where speed > 125;
```
```
UPDATE 13
```

```
select * from histogram('speed', 'bike');
```
```
 bucket |              range              |  freq  |       bar       
--------+---------------------------------+--------+-----------------
      1 | [0.0000000000,6.2172000000]     |  78499 | *
      2 | [6.2208000000,12.4380000000]    | 160721 | **
      3 | [12.4416000000,18.6600000000]   | 353664 | *****
      4 | [18.6624000000,24.8800000000]   | 713168 | ***********
      5 | [24.8832000000,31.1004000000]   | 981709 | ***************
      6 | [31.1012000000,37.3176000000]   | 630291 | **********
      7 | [37.3212000000,43.5384000000]   | 220459 | ***
      8 | [43.5420000000,49.7600000000]   |  66973 | *
      9 | [49.7628000000,55.9800000000]   |  19913 | 
     10 | [55.9836000000,62.2008000000]   |   5582 | 
     11 | [62.2044000000,68.4180000000]   |   1592 | 
     12 | [68.4612000000,74.6208000000]   |    444 | 
     13 | [74.6460000000,80.7660000000]   |    120 | 
     14 | [80.9244000000,87.0444000000]   |     63 | 
     15 | [87.1920000000,93.2076000000]   |     40 | 
     16 | [93.4128000000,98.9712000000]   |     29 | 
     17 | [99.8064000000,105.4150000000]  |     34 | 
     18 | [106.0880000000,111.9060000000] |    122 | 
     19 | [112.0140000000,118.0300000000] |     19 | 
     20 | [118.5080000000,124.3660000000] |     38 | 
     21 | [124.4020000000,124.4020000000] |      1 | 
```

#### Much better lets generate first derivative of speed.

```
with dev_list as (
    select round((speed_difference / time_difference),5) as deriv,
           time,
           workoutid
           from (
        select speed_difference,
               case when time_difference = 0 then 1 else time_difference end as time_difference,
               time,
               workoutid
               from (
            select speed - lag(speed) over (partition by workoutid order by time) as speed_difference,
                   time - lag(time) over (partition by workoutid order by time) as time_difference,
                   speed,
                   time,
                   workoutid
                from bike order by time)
        as foo)
    as bar
    order by workoutid,
             time )
update bike r1
  set speed_first = d1.deriv
  from dev_list as d1
  where d1.workoutid = r1.workoutid and
        d1.time = r1.time;
```

### Sanity check values
```
select * from histogram('speed_first', 'bike');
```
```
 bucket |          range          |  freq   |       bar       
--------+-------------------------+---------+-----------------
      1 | [-117.36700,-117.36700] |       1 | 
      9 | [-51.53400,-51.53400]   |       1 | 
     10 | [-43.03800,-35.98560]   |       5 | 
     11 | [-32.08680,-31.21440]   |       2 | 
     12 | [-24.46560,-19.06920]   |       8 | 
     13 | [-18.21960,-10.79280]   |      43 | 
     14 | [-10.56510,-2.58390]    |   11814 | 
     15 | [-2.58360,5.61240]      | 3214701 | ***************
     16 | [5.61600,13.66403]      |     404 | 
     17 | [14.04000,19.31960]     |      27 | 
     18 | [22.81680,29.28240]     |       7 | 
     21 | [46.60920,46.60920]     |       1 | 
```

#### (Re)Generate moving averages

##### Over 50 points
```
with dev_list as (
	select time, 
	       workoutid, 
	       avg(speed) over (partition by workoutid order by time rows between 50 preceding and current row) as mavg 
	from bike
	order by time
)
update bike r1 set speed_ma_50 = d1.mavg from dev_list as d1 where d1.workoutid = r1.workoutid and d1.time = r1.time;
```

```
select * from histogram('speed_ma_50', 'bike');
```
```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [0.00000,5.68744]     |   43081 | *
      2 | [5.68793,11.37565]    |   49392 | *
      3 | [11.37579,17.06365]   |  188611 | ***
      4 | [17.06372,22.75153]   |  573099 | ********
      5 | [22.75155,28.43940]   | 1127277 | ***************
      6 | [28.43944,34.12729]   |  931633 | ************
      7 | [34.12736,39.81515]   |  267330 | ****
      8 | [39.81522,45.50245]   |   46045 | *
      9 | [45.50322,51.19016]   |    5377 | 
     10 | [51.19624,56.86814]   |     913 | 
     11 | [56.88699,62.51061]   |     243 | 
     12 | [62.65249,68.24147]   |     129 | 
     13 | [68.26667,73.91977]   |      66 | 
     14 | [74.17537,79.56670]   |      67 | 
     15 | [79.81239,85.25330]   |      47 | 
     16 | [85.63583,90.91384]   |      21 | 
     17 | [91.34261,96.34795]   |      13 | 
     18 | [96.85913,101.81022]  |      23 | 
     19 | [102.45900,108.04700] |      17 | 
     20 | [108.08606,113.59865] |      96 | 
     21 | [113.75769,113.75769] |       1 | 
```

##### Over 100 points
```
with dev_list as (
	select time, 
	       workoutid, 
	       avg(speed) over (partition by workoutid order by time rows between 100 preceding and current row) as mavg 
	from bike
	order by time
)
update bike r1 set speed_ma_100 = d1.mavg from dev_list as d1 where d1.workoutid = r1.workoutid and d1.time = r1.time;
```

```
select * from histogram('speed_ma_100', 'bike');
```
```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [0.00000,5.59560]     |   41478 | *
      2 | [5.59620,11.19114]    |   37149 | 
      3 | [11.19135,16.78680]   |  159573 | **
      4 | [16.78685,22.38238]   |  530932 | *******
      5 | [22.38241,27.97799]   | 1150947 | ***************
      6 | [27.97802,33.57360]   | 1008843 | *************
      7 | [33.57364,39.16921]   |  262235 | ***
      8 | [39.16928,44.76440]   |   38581 | *
      9 | [44.76596,50.35773]   |    2542 | 
     10 | [50.37587,55.95134]   |     630 | 
     11 | [55.96428,61.54660]   |     199 | 
     12 | [61.55789,67.07230]   |     122 | 
     13 | [67.32360,72.28154]   |      52 | 
     14 | [73.11246,77.60127]   |       6 | 
     15 | [78.48609,83.75387]   |      28 | 
     16 | [84.35326,89.38780]   |      12 | 
     17 | [90.33991,94.41442]   |      25 | 
     18 | [95.17257,100.70780]  |      28 | 
     19 | [100.87944,106.02695] |      34 | 
     20 | [106.39543,111.82369] |      64 | 
     21 | [111.91205,111.91205] |       1 | 
```

## Altitude

### Big range of altitudes, some too low, some too high


```
select * from histogram('altitude', 'bike');
```
```
 bucket |                range                |  freq   |       bar       
--------+-------------------------------------+---------+-----------------
      1 | [-9999.0000000000,-9999.0000000000] |     501 | 
      9 | [-783.0000000000,134.9070000000]    | 8255274 | ***************
     10 | [134.9100000000,1260.8000000000]    | 2905586 | *****
     11 | [1260.9000000000,2386.8000000000]   |   97655 | 
     12 | [2386.9700000000,3512.4000000000]   |   10848 | 
     13 | [3523.2000000000,4141.4000000000]   |     532 | 
     14 | [4741.6000000000,5763.2000000000]   |    1042 | 
     15 | [5766.8000000000,6890.6000000000]   |     210 | 
     16 | [6891.0000000000,8011.8000000000]   |   15539 | 
     17 | [8021.2000000000,9140.2000000000]   |     154 | 
     18 | [9146.8000000000,10099.0000000000]  |     529 | 
     19 | [10323.4000000000,11394.6000000000] |    3367 | 
     20 | [11395.0000000000,12520.6000000000] |    2257 | 
     21 | [12520.8000000000,12520.8000000000] |       1 | 
```

#### Lets see how many rows are more thant two standard deviations  from the mean

```
select count(*) from (
	with dev_list as ( 
		select avg(altitude), stddev(altitude), workoutid from bike group by workoutid )
select altitude, r1.workoutid 
  from bike r1 
  join dev_list d1 on (d1.workoutid = r1.workoutid) 
  where r1.altitude < d1.avg - d1.stddev * 2 or r1.altitude > d1.avg + d1.stddev * 2) 
as foo;
```


#####  399,958 / 12,498,837  => ~3.2%
##### We should just smooth them to be two standard deviations from the mean

##### Fix values too low
```
with dev_list as ( 
	select avg(altitude), stddev(altitude), workoutid from bike group by workoutid )
update bike as r1 
  set altitude = d1.avg - (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.altitude < d1.avg - (d1.stddev * 2);
  ```
```
UPDATE 92719
```

##### Fix values too high
```
with dev_list as ( 
	select avg(altitude), stddev(altitude), workoutid from bike group by workoutid )
update bike as r1 
  set altitude = d1.avg + (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.altitude > d1.avg + (d1.stddev * 2);
```
```
UPDATE 312127
```

#### Sanity check

```
select * from histogram('altitude', 'bike');
```
```
 bucket |                range                |  freq   |       bar       
--------+-------------------------------------+---------+-----------------
      1 | [-9999.0000000000,-9999.0000000000] |     500 | 
      9 | [-912.7109130831,134.9070000000]    | 8260146 | ***************
     10 | [134.9100000000,1260.8000000000]    | 2900944 | *****
     11 | [1260.9000000000,2386.8000000000]   |   97453 | 
     12 | [2386.9700000000,3512.4000000000]   |   10801 | 
     13 | [3523.2000000000,3975.7592206068]   |     526 | 
     14 | [4885.6000000000,5763.2000000000]   |    1040 | 
     15 | [5766.8000000000,6890.6000000000]   |     229 | 
     16 | [6891.0000000000,8011.8000000000]   |   15551 | 
     17 | [8021.2000000000,9140.2000000000]   |     153 | 
     18 | [9150.6000000000,10161.2439623097]  |     531 | 
     19 | [10323.4000000000,11394.6000000000] |    3368 | 
     20 | [11395.0000000000,12520.6000000000] |    2252 | 
     21 | [12520.8000000000,12520.8000000000] |       1 | 
```

##### Everest is at 8,850 meters, and the most extreme high altitude ultra marathon only gets up to 5,300 meters. The lowest below sea level land is -413 meters. Probably a lot of errant sensor data

##### Lets look at workouts that include altitudes greater than 5300
```
select altitude, time, latitude, longitude, workoutid from bike where workoutid in (select distinct(workoutid) from bike where altitude > 5300) order by workoutid, time;
```

###### Small sample, look up lat / lng and check altitude. Spot check lat / lngs looks like the data is bad. 
```
     altitude     |    time    |   latitude    |   longitude   | workoutid 
------------------+------------+---------------+---------------+-----------
  7360.2951948472 | 1388210080 | 55.6914129000 | 12.5008919000 | 280192381
  6892.8000000000 | 1388210090 | 55.6917490000 | 12.5015103000 | 280192381
  6892.8000000000 | 1388210114 | 55.6926525000 | 12.5038258000 | 280192381
  6892.8000000000 | 1388210126 | 55.6931065000 | 12.5049203000 | 280192381
  6892.8000000000 | 1388210140 | 55.6934101000 | 12.5061636000 | 280192381
  6892.8000000000 | 1388210174 | 55.6933934000 | 12.5063811000 | 280192381
  6892.8000000000 | 1388210197 | 55.6941351000 | 12.5084967000 | 280192381
  6892.8000000000 | 1388210218 | 55.6949980000 | 12.5104137000 | 280192381
  6892.8000000000 | 1388210228 | 55.6953771000 | 12.5113158000 | 280192381
  6892.8000000000 | 1388210246 | 55.6963540000 | 12.5126180000 | 280192381
  6892.8000000000 | 1388210257 | 55.6969317000 | 12.5135342000 | 280192381
```

###### How many workouts is it?
```
select count(*) from  (select distinct(workoutid) from bike where altitude > 5300) as foo;
```

###### 46 / 25,000.
```
select count(*) from bike where altitude > 5300;
```
##### 22,625 probably should just drop the altitude for those workouts, as that is ~491 per workout.
```
update bike set altitude = NULL where workoutid in (select distinct(workoutid) from bike where altitude > 5300);
```
```
UPDATE 22996
```

##### (Re)Generate first derivative
```
with dev_list as (
    select round((alt_difference / time_difference),5) as deriv, time, workoutid from ( 
        select alt_difference, case when time_difference = 0 then 1 else time_difference end as time_difference, time, workoutid from (
            select altitude - lag(altitude) over (partition by workoutid order by time) as alt_difference, time - lag(time) over (partition by workoutid order by time) as time_difference, time, workoutid from bike order by time) 
        as foo) 
    as bar order by workoutid, time )
update bike r1 
  set altitude_first = d1.deriv 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and d1.time = r1.time;
```


##### (Re)Generate second derivative
```
with dev_list as (
    select round((alt_difference / time_difference),5) as deriv, time, workoutid from ( 
        select alt_difference, case when time_difference = 0 then 1 else time_difference end as time_difference, time, workoutid from (
            select altitude_first - lag(altitude_first) over (partition by workoutid order by time) as alt_difference, time - lag(time) over (partition by workoutid order by time) as time_difference, time, workoutid from bike order by time) 
        as foo) 
    as bar order by workoutid, time )
update bike r1 
  set altitude_first = d1.deriv 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and d1.time = r1.time;
```


#### Vacuum
```
vacuum full bike;
```

## Elapsed distance


```
with elap_dist as (select time,
       r1.workoutid,
       (row_number() over (partition by r1.workoutid order by time) - 1) * (distance / (series_length - 1)) as elapsed_distance
       from bike r1 join bike_by_workout r2 on (r1.workoutid = r2.workoutid) order by time)   
update bike as r1 
  set elapsed_distance = round(d1.elapsed_distance,10)
  from elap_dist as d1 
  where d1.workoutid = r1.workoutid and d1.time = r1.time;
```


## Elapsed time

```
with elap_time as (select time, workoutid, time - first_value(time) over (partition by workoutid order by time) as time_elap from bike order by time)
update bike as r1 
  set elapsed_time = d1.time_elap 
  from elap_time as d1 
  where r1.workoutid = d1.workoutid and r1.time = d1.time;
```


## Heart Rate

```
select * from histogram('heart_rate', 'bike');
```
```
 bucket |         range         |  freq   |       bar
--------+-----------------------+---------+-----------------
      1 | [0.00000,15.00000]    |   32432 |
      2 | [26.00000,45.00000]   |    1165 |
      3 | [46.00000,68.00000]   |   18548 |
      4 | [69.00000,90.00000]   |  259807 | *
      5 | [91.00000,113.00000]  | 1429892 | ******
      6 | [114.00000,136.00000] | 3383049 | ***************
      7 | [137.00000,159.00000] | 3236420 | **************
      8 | [160.00000,181.00000] |  955914 | ****
      9 | [182.00000,204.00000] |   71817 |
     10 | [205.00000,227.00000] |    5739 |
     11 | [228.00000,250.00000] |    3192 |
     12 | [251.00000,269.00000] |     172 |
     13 | [281.00000,295.00000] |       2 |
     14 | [305.00000,305.00000] |       1 |
     15 | [319.00000,341.00000] |       3 |
     16 | [353.00000,363.00000] |       2 |
     17 | [375.00000,375.00000] |       1 |
     18 | [387.00000,399.00000] |       2 |
     20 | [441.00000,441.00000] |       1 |
     21 | [455.00000,455.00000] |       1 |
```

##### Lots of garbage data. Heart rate should never be above 240 or below 40. Even those are uncommon. Use the same two standard deviation logic as altitude. 

##### Fix values too high
```
with dev_list as ( 
	select avg(heart_rate), stddev(heart_rate), workoutid from bike group by workoutid )
update bike as r1 
  set heart_rate = d1.avg + (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.heart_rate > d1.avg + (d1.stddev * 2);
```
```
UPDATE 100283
```

##### Fix values too low
```
with dev_list as ( 
	select avg(heart_rate), stddev(heart_rate), workoutid from bike group by workoutid )
update bike as r1 
  set heart_rate = d1.avg + (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.heart_rate < d1.avg - (d1.stddev * 2);
```
```
UPDATE 307317
```

#### Better, but still too many below 40 and above 250
```
select * from histogram('heart_rate', 'bike');
```
```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [0.00000,1.00000]     |   31399 | 
      2 | [15.00000,26.00000]   |     501 | 
      3 | [29.00000,43.00000]   |     804 | 
      4 | [44.00000,57.00000]   |    1802 | 
      5 | [58.00000,71.00000]   |   14350 | 
      6 | [72.00000,86.17740]   |  104212 | *
      7 | [86.46352,100.49434]  |  404999 | ***
      8 | [100.71122,114.90386] | 1045923 | *******
      9 | [114.93943,129.29855] | 2045875 | *************
     10 | [129.30768,143.66779] | 2409810 | ***************
     11 | [143.67269,158.03933] | 2035927 | *************
     12 | [158.04285,172.40586] |  954062 | ******
     13 | [172.40803,186.76659] |  285282 | **
     14 | [186.77480,201.13263] |   51426 | 
     15 | [201.14893,215.28694] |    7480 | 
     16 | [215.78432,229.53088] |    2071 | 
     17 | [230.00000,244.00000] |    1846 | 
     18 | [244.68008,254.00000] |     294 | 
     19 | [259.58018,270.41792] |      90 | 
     21 | [287.34468,287.34468] |       7 | 
```

#### (Re)Generate moving heart rate average over last 25 points

```
with dev_list as (
	select time,
	       workoutid, 
	       avg(heart_rate) over (partition by workoutid order by time rows between 25 preceding and current row) as mavg 
	from bike 
	order by time
)
update bike r1 set heart_rate_ma_25 = d1.mavg from dev_list as d1 where d1.workoutid = r1.workoutid and d1.time = r1.time;
```

```
select * from histogram('heart_rate_ma_25', 'bike');
```

```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [0.00000,14.00000]    |   31248 | 
      2 | [14.70000,28.30769]   |     538 | 
      3 | [28.76923,43.07692]   |     655 | 
      4 | [43.11538,57.46580]   |    1501 | 
      5 | [57.50000,71.83333]   |    8432 | 
      6 | [71.83636,86.20338]   |   58022 | 
      7 | [86.20414,100.57036]  |  288718 | **
      8 | [100.57083,114.93784] |  957449 | *****
      9 | [114.93788,129.30510] | 2067928 | ***********
     10 | [129.30513,143.67230] | 2830357 | ***************
     11 | [143.67237,158.03937] | 2088621 | ***********
     12 | [158.03969,172.40615] |  843053 | ****
     13 | [172.40686,186.77387] |  188661 | *
     14 | [186.77480,201.13464] |   25883 | 
     15 | [201.14374,215.50000] |    4005 | 
     16 | [215.52157,229.85714] |    1629 | 
     17 | [229.88195,244.19231] |    1325 | 
     18 | [244.26923,254.71663] |      92 | 
     19 | [259.53093,270.41792] |      36 | 
     21 | [287.34468,287.34468] |       7 | 
```